<a href="https://colab.research.google.com/github/BennoKrojer/Probe-Masked-LMs/blob/master/probeBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install transformers
!pip install torch

     |████████████████████████████████| 573kB 4.9MB/s 
     |████████████████████████████████| 890kB 61.8MB/s 
     |████████████████████████████████| 3.7MB 46.6MB/s 
     |████████████████████████████████| 1.0MB 47.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=b64fe9c0fc03ac94804fb91cc4440d520fe65e6c0670bd0838d88f4757377565
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
probes_raw = """
nice is a synonym of
good is a synonym of
bad is a synonym of
sweet is a synonym of
red is a synonym of
pretty is a synonym of
beautiful is a synonym of
friendly is a synonym of
usual is a synonym of
common is a synonym of
ordinary is a synonym of
"""
oneto1 = False

In [0]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, RobertaForMaskedLM, RobertaTokenizer
import numpy as np

# from: https://huggingface.co/transformers/quickstart.html#bert-example

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForMaskedLM.from_pretrained('bert-large-uncased')
model.eval()

previous_preds = []
previous_ent = ''
both_correct = ''
both_correct_c = 0
both_false = ''
both_false_c = 0
asymmetric_false = ""
asymmetric_c = 0
rest = []
probes = probes_raw.split('\n')
for i, probe in enumerate(probes):
  if len(probe) > 5:
    text = f'[CLS] {probe} [MASK] . [SEP]'
    tokenized_text = tokenizer.tokenize(text)
    masked_index = [i for i, x in enumerate(tokenized_text) if x == '[MASK]'][0]
    # print(tokenized_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [0]*len(tokenized_text)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor, token_type_ids=segments_tensors)
        predictions = outputs[0][0][masked_index]
    predicted_ids = torch.argsort(predictions, descending=True)[:20]
    predicted_tokens = tokenizer.convert_ids_to_tokens(list(predicted_ids))
    if oneto1:
      if i % 2 == 1:
        previous_preds = predicted_tokens
        previous_ent = probe.split("'")[0]
      else:
        current_ent = probe.split()[0]
        current_prediction = predicted_tokens[0] if predicted_tokens[0] != current_ent else predicted_tokens[1] 
        prev_prediction = previous_preds[0] if previous_preds[0] != previous_ent else previous_preds[1]
        if current_ent == prev_prediction and current_prediction == previous_ent:
          both_correct += f'{probes[i-1]} [MASK] → {previous_preds}\n{probe} [MASK] → {predicted_tokens}\n'
          both_correct_c += 1
        elif current_ent != prev_prediction and current_prediction != previous_ent:
          both_false += f'{probes[i-1]} [MASK] → {previous_preds}\n{probe} [MASK] → {predicted_tokens}\n'
          both_false_c += 1
        else:
          asymmetric_false += f'{probes[i-1]} [MASK] → {previous_preds}\n{probe} [MASK] → {predicted_tokens}\n'
          asymmetric_c += 1
    else:
      print(f'{probe} [MASK] → {predicted_tokens}')

if oneto1:
  print(both_correct_c)
  print(both_false_c)
  print(asymmetric_c)
  print('both correct:')
  print(both_correct)
  print('both false')
  print(both_false)
  print('asymmetrically false')
  print(asymmetric_false)


    # print(f'{probe} [MASK] → {predicted_tokens}')



nice is a synonym of [MASK] → ['nice', 'it', 'var', 'ana', 'io', 'cicero', 'minor', 'bella', 'venus', '[UNK]', 'ca', 'this', 'normal', 'homo', 'c', 'p', 'mons', 'florida', 'pseudo', 'nova']
good is a synonym of [MASK] → ['good', 'bad', 'excellent', 'perfect', 'great', 'superb', 'nice', 'gold', 'best', 'beautiful', 'fair', 'sir', 'fine', 'poor', 'evil', 'b', 'it', 'venus', 'minor', 'p']
bad is a synonym of [MASK] → ['good', 'bad', 'big', 'sir', 'nice', 'mad', 'it', 'red', 'b', 'black', 'la', 'pro', 'great', 'gold', 'perfect', 'old', 'no', 'excellent', 'vice', 'negro']
sweet is a synonym of [MASK] → ['sweet', 'red', 'black', 'rose', 'white', 'gray', 'brown', 'good', 'blue', 'love', 'yellow', 'bird', '[UNK]', 'kali', 'jack', 'sugar', 'honey', 'bitter', 'may', 'winter']
red is a synonym of [MASK] → ['black', 'blue', 'yellow', 'red', 'white', 'green', 'brown', 'orange', 'gold', 'purple', 'grey', 'gray', 'violet', 'pink', 'dark', 'silver', 'rose', 'ruby', 'pale', 'sapphire']
pretty is a syno